# Centroiding

The centroid of an object refers to its center of light (similar to center of mass, except we mean the center of the point spread function instead of a mass distribution). Centroiding therefore refers to finding the center of a particular source, generally so that we can accurately place a photometric aperture around it. 

Photutils has a convenient module dedicated to finding centroids (`photutils.centroids`). 

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from photutils.datasets import load_star_image
from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture
from photutils.centroids import centroid_sources
import photutils.centroids as centroids

from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.stats import sigma_clipped_stats, SigmaClip

## First, identify sources

This is identical to notebook 01-02

In [ ]:
# photutils allows us to easily access one of their example images
hdu = load_star_image()  
# We will only consider a small portion of the image, for faster processing
data = hdu.data[0:401, 0:401]  

# Plot the image with a square-root normalization
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, norm=norm, origin='lower', cmap='Greys_r')

In [ ]:
# Use sigma-clipped statistics to estimate the background & background noise
clipped_mean, clipped_med, clipped_std = sigma_clipped_stats(data, sigma=3.0) 
print(clipped_mean, clipped_med, clipped_std)

# Identify sources
daofind = DAOStarFinder(fwhm=3.0, threshold=5.*clipped_std)  
sources = daofind(data - clipped_med)

Note that the source finder already identifies a centroid for each object, and gives us the x and y coordinate of that centroid

In [ ]:
for col in sources.colnames:  
    if col not in ('id', 'npix'):
        sources[col].info.format = '%.2f'  # for consistent table output
sources.pprint(max_width=76)  

In [ ]:
sources.show_in_notebook()

However, we don't always need or want to run a full source-finding analysis on every image we work with. Sometimes we know the location of our target star already, or at least the approximate location.

## Centroid of a single star

For example, let's consider the bright star around x=300 and y=95 in our original image. `photutils.centroids` offers us several different ways to calculate the centroid of a given object:

* `centroid_com`: Calculates the object
  "center of mass" from 2D image moments.

* `centroid_quadratic`: Calculates the
  centroid by fitting a 2D quadratic polynomial to the data.

* `centroid_1dg`: Calculates the centroid
  by fitting 1D Gaussians to the marginal ``x`` and ``y``
  distributions of the data.

* `centroid_2dg`: Calculates the centroid
  by fitting a 2D Gaussian to the 2D distribution of the data.

In [ ]:
# First, subtract the background from the whole image
bkg_sub_data = data - clipped_med
# Then trim just to the region of interest
subregion = bkg_sub_data[75:115,280:320]

In [ ]:
# centroid_com Calculates the object "center of mass" from 2D image moments.
# We don't give it any initial guesses, it's just going to find the weighted center
# of all the light in the image
x1, y1 = centroids.centroid_com(subregion)
print(x1,y1)

In [ ]:
# centroid_com Calculates the centroid of an n-dimensional array by fitting a 2D quadratic polynomial.
# We can provide it an initial guess for its fit, or let it use the brightest pixel as the starting point
x2, y2 = centroids.centroid_quadratic(subregion,xpeak=21,ypeak=20,fit_boxsize=13,search_boxsize=3)
print(x2,y2)

In [ ]:
# centroid_com Calculates the centroid of a 2D array by fitting 1D Gaussians 
# to the marginal x and y distributions of the array.
# We don't give it any initial guesses, it's just going to find the center of the brightest thing there
x3, y3 = centroids.centroid_1dg(subregion)
print(x3,y3)

In [ ]:
# centroid_com Calculates the centroid of a 2D array by fitting a 2D gaussian to x and y together
# We don't give it any initial guesses, it's just going to find the center of the brightest thing there
x4, y4 = centroids.centroid_2dg(subregion)
print(x4,y4)

In [ ]:
# Plot the image with a square-root normalization
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(subregion, norm=norm, origin='lower', cmap='Greys_r')
plt.plot(x1,y1,'+')
plt.plot(x2,y2,'+')
plt.plot(x3,y3,'+')
plt.plot(x4,y4,'+')

The blue cross (from the center-of-mass calculation) is clearly offset because there's another source in the image. This is similar to the center of mass of a binary system, which is not at the center of the more massive object, but offset along the line between the two objects.

The quadratic fit is closer to the center of the brighter object (our target), but a little offset towards the fainter one. 

The other two guesses are very close, and basically on top of each other. Since we are looking at stars which ideally have round images on the CCD, the choice of fitting 1d or 2d Gaussians doesn't make much of a difference. If we were working with galaxies or if there are significant distortions, a 2d Gaussian might produce a better solution. 

## Finding the centroids of multiple sources at once

Very rarely in astronomy do we actually study only a single star in an image! Therefore, we will want to find the positions of many stars at once. For this, we use the `centroid_sources` function. This function requires an image, initial x and y guesses for each star, the number of pixels around each star to consider, and a choice of centroiding function. 

We'll return to our full example image (background subtracted) for this one. 

In [ ]:
x_init = np.array([110, 270, 152, 84])
y_init = np.array([130, 220, 275, 244])

In [ ]:
x_all, y_all = centroid_sources(bkg_sub_data, x_init, y_init, box_size=13, centroid_func=centroids.centroid_2dg)

In [ ]:
for x, y in zip(x_all,y_all):
    print(x,y)

In time-series photometry, we want to find the position of our target and one (or more) comparison stars in every image we take. Ideally, each star will stay in roughly the same position from exposure to exposure. However, it likely won't be stable to within 1/100th of a pixel. 

Therefore, we don't need to run source-finding in every image of a series. You will want to use the position in one image as the initial guess for the position in the next image.